In [1]:
from nba_api.stats.endpoints import leaguegamelog
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from termcolor import colored
import requests
import mysql.connector
import csv_getters.helper_functions as hf
from csv_getters.config import api_basketball_key, conn_host, conn_database, conn_user, conn_password
import datetime as dt
from IPython.display import clear_output
from dateutil import tz
import json
import time
from nba_api.stats.static import teams 
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as soup
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from statistics import mean
import urllib.request

In [2]:
season = '2022-2023'
season_split = int(season.split('-')[0])
now =  time.time()
date = dt.datetime.now().strftime('%Y-%m-%d')

min_threshold = 1.8
max_threshold = 10

totals_max_threshold = 55
totals_min_threshold = 45

min_games = 8
totals_n_last_games = 15
n_last_games = 10
n_last_specific_games = 5

from_zone = tz.gettz('UTC')
to_zone = tz.gettz('America/Sao_Paulo')

pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
def connect_to_db():
    return mysql.connector.connect(host=conn_host, 
                                     database=conn_database,
                                     user=conn_user,
                                     password=conn_password)

def execute_query(query, read_only = True):
    resp = None
    try:
        db = connect_to_db()
        if read_only:
            resp = pd.read_sql_query(query, db)
        else:
            mycursor = db.cursor()
            mycursor.execute(query)

            db.commit()
    except Exception as e:
        print(e)
    db.close()
    return resp

def execute_multiple_queries(queries):
    try:
        db = connect_to_db()
        mycursor = db.cursor()
        for query in queries:
            mycursor.execute(query)

        db.commit()
        db.close()
    except Exception as e:
        print(e)

In [4]:
def add_match_info_to_db(home_game, away_game, winner):
    query = (f"INSERT IGNORE INTO games (id, date, season, is_playoff, winner, " + 
                  "home_id, home_pts, home_fgm, home_fga, home_fg_pct, home_fg3m, home_fg3a, home_fg3_pct, home_ftm, home_fta, home_ft_pct, home_oreb, home_dreb, home_reb, home_ast, home_stl, home_blk, home_tov, home_pf," +
                  "away_id, away_pts, away_fgm, away_fga, away_fg_pct, away_fg3m, away_fg3a, away_fg3_pct, away_ftm, away_fta, away_ft_pct, away_oreb, away_dreb, away_reb, away_ast, away_stl, away_blk, away_tov, away_pf" +
                  f") VALUES ({home_game['GAME_ID']}, '{home_game['GAME_DATE']}', {get_season_year(home_game['SEASON_ID'])}, {home_game['IS_PLAYOFFS']}, '{winner}', " + 
                  f"{home_game['TEAM_ID']}, {home_game['PTS']}, {home_game['FGM']}, {home_game['FGA']}, {home_game['FG_PCT']}, {home_game['FG3M']}, {home_game['FG3A']}, {home_game['FG3_PCT']}, {home_game['FTM']}, {home_game['FTA']}, {home_game['FT_PCT']}, {home_game['OREB']}, {home_game['DREB']}, {home_game['REB']}, {home_game['AST']}, {home_game['STL']}, {home_game['BLK']}, {home_game['TOV']}, {home_game['PF']}, " +
                  f"{away_game['TEAM_ID']}, {away_game['PTS']}, {away_game['FGM']}, {away_game['FGA']}, {away_game['FG_PCT']}, {away_game['FG3M']}, {away_game['FG3A']}, {away_game['FG3_PCT']}, {away_game['FTM']}, {away_game['FTA']}, {away_game['FT_PCT']}, {away_game['OREB']}, {away_game['DREB']}, {away_game['REB']}, {away_game['AST']}, {away_game['STL']}, {away_game['BLK']}, {away_game['TOV']}, {away_game['PF']}" +
                  ")")
    return query

def add_player_to_db(player_id, name):
    query = (f"INSERT IGNORE INTO players (id, name) VALUES ({player_id}, '{name}')")
    return query
    
def add_player_game_to_db(game):
    query = (f"INSERT IGNORE INTO playergames (team_id, player_id, game_id, minutes, pts, fgm, fga, fg_pct, fg3m, fg3a, fg3_pct, ftm, fta, ft_pct, oreb, dreb, reb, ast, stl, blk, tov, pf, plus_minus) " + 
                  f"VALUES ({game['TEAM_ID']}, {game['PLAYER_ID']}, {game['GAME_ID']}, {game['MIN']}, {game['PTS']}, {game['FGM']}, {game['FGA']}, {game['FG_PCT']}, {game['FG3M']}, {game['FG3A']}, {game['FG3_PCT']}, {game['FTM']}, {game['FTA']}, {game['FT_PCT']}, {game['OREB']}, {game['DREB']}, {game['REB']}, {game['AST']}, {game['STL']}, {game['BLK']}, {game['TOV']}, {game['PF']}, {game['PLUS_MINUS']})")
    return query

def get_season(season, season_type='Regular Season'):
    season_games = leaguegamelog.LeagueGameLog(season = str(season), season_type_all_star=season_type).get_data_frames()[0]
    season_players = leaguegamelog.LeagueGameLog(season = str(season), player_or_team_abbreviation = 'P', season_type_all_star=season_type).get_data_frames()[0]
    season_games['IS_PLAYOFFS'] = True if season_type == 'Playoffs' else False
    season_players['IS_PLAYOFFS'] = True if season_type == 'Playoffs' else False
    
    season_games.dropna(subset=['FG_PCT','FT_PCT','FG3_PCT'], inplace=True)

    season_games['GAME_ID'] = pd.to_numeric(season_games['GAME_ID'])
    season_players['GAME_ID'] = pd.to_numeric(season_players['GAME_ID'])

    season_games['GAME_DATE'] = pd.to_datetime(season_games['GAME_DATE'])
    season_players['GAME_DATE'] = pd.to_datetime(season_players['GAME_DATE'])

    season_games = season_games.sort_values(['GAME_DATE', 'GAME_ID'], ascending=[True, True]).reset_index(drop=True)
    return season_games, season_players

def get_season_year(season_id):
    return int(str(season_id)[1:])

In [5]:
season_games, season_players = get_season(season_split, season_type='Regular Season')

In [6]:
season_games.tail()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE,IS_PLAYOFFS
489,22022,1610612745,HOU,Houston Rockets,22200245,2022-11-20,HOU vs. GSW,L,240,41,...,45,24,13,1,14,12,120,-7,1,False
490,22022,1610612742,DAL,Dallas Mavericks,22200246,2022-11-20,DAL vs. DEN,L,240,35,...,36,24,5,3,13,18,97,-1,1,False
491,22022,1610612743,DEN,Denver Nuggets,22200246,2022-11-20,DEN @ DAL,W,240,37,...,44,22,10,5,16,20,98,1,1,False
492,22022,1610612747,LAL,Los Angeles Lakers,22200247,2022-11-20,LAL vs. SAS,W,240,45,...,54,29,5,2,15,10,123,31,1,False
493,22022,1610612759,SAS,San Antonio Spurs,22200247,2022-11-20,SAS @ LAL,L,240,41,...,44,26,7,2,10,17,92,-31,1,False


In [7]:
season_players.tail()

,SEASON_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,...,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,FANTASY_PTS,VIDEO_AVAILABLE,IS_PLAYOFFS
5316,22022,1630227,Daishen Nix,1610612745,HOU,Houston Rockets,22200245,2022-11-20,HOU vs. GSW,L,...,5,1,0,1,0,0,8,10.7,1,False
5317,22022,1628978,Donte DiVincenzo,1610612744,GSW,Golden State Warriors,22200245,2022-11-20,GSW @ HOU,W,...,1,1,1,1,1,4,-7,15.3,1,False
5318,22022,201569,Eric Gordon,1610612745,HOU,Houston Rockets,22200245,2022-11-20,HOU vs. GSW,L,...,2,1,0,2,0,14,-1,20.4,1,False
5319,22022,1631246,Vince Williams Jr.,1610612763,MEM,Memphis Grizzlies,22200243,2022-11-20,MEM @ BKN,L,...,0,0,0,0,0,2,5,2.0,1,False
5320,22022,1630559,Austin Reaves,1610612747,LAL,Los Angeles Lakers,22200247,2022-11-20,LAL vs. SAS,W,...,3,0,0,1,1,21,19,26.9,1,False


In [8]:
games_to_insert_queries = []
players_to_insert_queries = []
player_games_to_insert_queries = []

season_id = ''
for i, g in season_games.groupby(season_games.index // 2):
    clear_output(wait=True)
    print("{}/{}".format(i, len(season_games.index) // 2))
    if g.iloc[[0],:].iloc[0]['WL'] == None:
        break
        
    if '@' in g.iloc[[0],:].iloc[0]['MATCHUP']:
        away_game = g.iloc[0,:]
        home_game = g.iloc[1,:]
        winner = 'H' if g.iloc[1,:]['WL'] == 'W' else 'A'
    else:
        home_game = g.iloc[0,:]
        away_game = g.iloc[1,:]
        winner = 'H' if g.iloc[0,:]['WL'] == 'W' else 'A'
    
    game_players = season_players.loc[season_players['GAME_ID'] == home_game['GAME_ID']]
    game_players = game_players.replace({np.nan: 0})
    
    games_to_insert_queries.append(add_match_info_to_db(home_game, away_game, winner))
    
    for index, player in game_players.iterrows():
        players_to_insert_queries.append(add_player_to_db(player['PLAYER_ID'], player['PLAYER_NAME'].replace("'", "")))
        player_games_to_insert_queries.append(add_player_game_to_db(player))

246/247


In [9]:
execute_multiple_queries(games_to_insert_queries)
execute_multiple_queries(players_to_insert_queries)
execute_multiple_queries(player_games_to_insert_queries)

In [10]:
teams_list = teams.get_teams()
teams_df = pd.DataFrame(teams_list)

In [11]:
teams_df.head()

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Atlanta,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966


In [12]:
season_games = execute_query(f"SELECT g.id, g.date, g.season, g.is_playoff, g.winner, g.home_id, ht.name as home_name, g.home_pts, g.home_fgm, g.home_fga, g.home_fg_pct, g.home_fg3m, g.home_fg3a, g.home_fg3_pct, g.home_ftm, g.home_fta, g.home_ft_pct, g.home_oreb, g.home_dreb, g.home_reb, g.home_ast, g.home_stl, g.home_blk, g.home_tov, g.home_pf, g.away_id, at.name as away_name, g.away_pts, g.away_fgm, g.away_fga, g.away_fg_pct, g.away_fg3m, g.away_fg3a, g.away_fg3_pct, g.away_ftm, g.away_fta, g.away_ft_pct, g.away_oreb, g.away_dreb, g.away_reb, g.away_ast, g.away_stl, g.away_blk, g.away_tov, g.away_pf, g.home_odds, g.away_odds, g.over_under_line, g.spread_line FROM games AS g LEFT JOIN teams as ht ON g.home_id = ht.id LEFT JOIN teams as at ON g.away_id = at.id WHERE g.season >= {season_split - 5} and g.season <= {season_split} ORDER BY g.date ASC")
season_games_plyrs = execute_query(f"SELECT g.id as game_id, g.date, g.season, g.is_playoff, g.winner, g.home_id, g.away_id, pg.team_id, p.name as player_name , pg.player_id, pg.minutes, pg.pts, pg.fgm, pg.fga, pg.fg_pct, pg.fg3m, pg.fg3a, pg.fg3_pct, pg.ftm, pg.fta, pg.ft_pct, pg.oreb, pg.dreb, pg.reb, pg.ast, pg.stl, pg.blk, pg.tov, pg.pf, pg.plus_minus FROM playergames AS pg LEFT JOIN games as g on pg.game_id = g.id LEFT JOIN players as p on pg.player_id = p.id WHERE g.season >= {season_split - 5} and g.season = {season_split} ORDER BY g.date ASC")
season_games_plyrs['player_name'] = season_games_plyrs['player_name'].apply(lambda x: x.replace('Jr.', '').strip())

C:\Users\caiog\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\caiog\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [13]:
season_games_plyrs.head()

,game_id,date,season,is_playoff,winner,home_id,away_id,team_id,player_name,player_id,...,ft_pct,oreb,dreb,reb,ast,stl,blk,tov,pf,plus_minus
0,22200001,2022-10-18,2022,0,H,1610612738,1610612755,1610612755,P.J. Tucker,200782,...,0.0,2,2,4,0,0,1,2,2,-6
1,22200001,2022-10-18,2022,0,H,1610612738,1610612755,1610612738,Al Horford,201143,...,0.0,1,4,5,1,0,0,0,4,8
2,22200001,2022-10-18,2022,0,H,1610612738,1610612755,1610612738,Blake Griffin,201933,...,0.5,2,3,5,1,0,0,0,3,-5
3,22200001,2022-10-18,2022,0,H,1610612738,1610612755,1610612755,James Harden,201935,...,1.0,0,8,8,7,0,0,3,3,1
4,22200001,2022-10-18,2022,0,H,1610612738,1610612755,1610612755,Tobias Harris,202699,...,0.5,1,1,2,0,3,0,0,3,-1


In [14]:
season_games['home_off_rtg'] = season_games.apply(lambda row: hf.get_team_offensive_rating_game(row, 'H'), axis = 1)
season_games['home_def_rtg'] = season_games.apply(lambda row: hf.get_team_offensive_rating_game(row, 'H'), axis = 1)

season_games['away_off_rtg'] = season_games.apply(lambda row: hf.get_team_offensive_rating_game(row, 'A'), axis = 1)
season_games['away_def_rtg'] = season_games.apply(lambda row: hf.get_team_offensive_rating_game(row, 'A'), axis = 1)

In [15]:
season_games.head()

,id,date,season,is_playoff,winner,home_id,home_name,home_pts,home_fgm,home_fga,...,away_tov,away_pf,home_odds,away_odds,over_under_line,spread_line,home_off_rtg,home_def_rtg,away_off_rtg,away_def_rtg
0,21700001,2017-10-17,2017,0,H,1610612739,Cleveland Cavaliers,102,38,83,...,12,24,1.52,2.62,216.0,4.5,100.990099,100.990099,98.019802,98.019802
1,21700002,2017-10-17,2017,0,A,1610612744,Golden State Warriors,121,43,80,...,13,16,1.22,4.49,231.5,9.0,121.730382,121.730382,113.382900,113.382900
2,21700003,2017-10-18,2017,0,H,1610612765,Detroit Pistons,102,41,96,...,17,15,1.67,2.27,202.0,2.5,101.190476,101.190476,91.277890,91.277890
3,21700004,2017-10-18,2017,0,H,1610612754,Indiana Pacers,140,53,102,...,20,25,1.65,2.30,216.5,3.0,121.951220,121.951220,113.126079,113.126079
4,21700005,2017-10-18,2017,0,H,1610612753,Orlando Magic,116,43,90,...,13,20,2.41,1.60,205.5,3.5,109.848485,109.848485,99.271403,99.271403


In [16]:
def get_season_fixtures(season):
    headers = {
        'X-RapidAPI-Key': api_basketball_key,
        'X-RapidAPI-Host': 'api-basketball.p.rapidapi.com'
    }
    response = requests.get(f"https://api-basketball.p.rapidapi.com/games?league=12&season={season}&timezone=America/Sao_Paulo", headers=headers)
    response_parsed = json.loads(response.text)
    return response_parsed['response']

In [17]:
fixtures = get_season_fixtures(season)
fixtures = [fixture for fixture in fixtures if fixture['timestamp'] >= now and fixture['timestamp'] <= (now + 24*60*60)]
fixtures = sorted(fixtures, key = lambda x: x['timestamp'])

In [18]:
from joblib import dump, load

save_current_season_model = True
current_season = 2022

model = load('model.joblib')

In [19]:
def scrape_bookies_data():
    base_url = f"https://www.pinnacle.com/pt/basketball/nba/matchups#period:0"
    option = Options()
#     option.headless = True
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=option)
    driver.get(base_url)
    driver.maximize_window()
    time.sleep(5)
    
    element = driver.find_element('xpath', '//*[@id="root"]/div/div[2]/main/div/div[4]/div[2]/div')
    htmlContent = element.get_attribute('outerHTML')
    page_soup = soup(htmlContent, "html.parser")
    
    games = page_soup.findAll('div', {"class": "style_row__21s9o style_row__21_Wa"})
    
    bookies_info = dict()
    
    print(games)
    for g in games:
        try:
            teams = g.findAll('span', {"class": "ellipsis event-row-participant style_participant__H8-ku"})
            home_team = teams[1].text
            away_team = teams[0].text
                        
            home = teams_df.loc[teams_df['full_name'] == home_team].iloc[0]
            away = teams_df.loc[teams_df['full_name'] == away_team].iloc[0]

            bets = g.findAll('div', {"class": "style_buttons__XEQem"})
            
            spread_line = bets[0].findAll('span', {"class": "style_label__2KJur"})[0].text[1:]

            ml_prices = bets[1].findAll('span', {"class": "style_price__15SlF"})
            home_ml_odds = ml_prices[1].text
            away_ml_odds = ml_prices[0].text

            totals_line = bets[2].findAll('span', {"class": "style_label__2KJur"})[0].text
            totals_prices = bets[2].findAll('span', {"class": "style_price__15SlF"})

            totals_over_odds = totals_prices[0].text
            totals_under_odds = totals_prices[1].text

            print(f"\n{home_team} x {away_team}")
            print(f"ML Odds: {home_ml_odds} // {away_ml_odds}")
            print(f"Totals Line: {totals_line}")
            print(f"Spread Line: {spread_line}")
            print(f"Over/Under Odds: {totals_over_odds} // {totals_under_odds}")
            
            bookies_info[f"{home['id']}x{away['id']}"] = {
                'home_ml_odds': float(home_ml_odds),
                'away_ml_odds': float(away_ml_odds),
                'totals_line': float(totals_line),
                'spread_line': float(spread_line),
                'totals_over_odds': float(totals_over_odds),
                'totals_under_odds': float(totals_under_odds)
            }
        except Exception as e:
            print(e)
            continue
        
    driver.quit()
    return bookies_info

In [20]:
bookies_info = scrape_bookies_data()

C:\Users\caiog\AppData\Local\Temp\ipykernel_5192\1201892260.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=option)


[<div class="style_row__21s9o style_row__21_Wa"><div class="style_metadata__DPrQU"><div><a class="" href="/pt/basketball/nba/orlando-magic-vs-indiana-pacers/1563197100"><div class="flex-row style_gameInfo__1mTg3"><div class="style_matchupMetadata__39RAC"><div class="ellipsis style_gameInfoLabel__24vcV"><span class="ellipsis event-row-participant style_participant__32s23">Orlando Magic</span></div><div class="ellipsis style_gameInfoLabel__24vcV"><span class="ellipsis event-row-participant style_participant__32s23">Indiana Pacers</span></div><div class="style_matchupDate__3maWe"><span>21:00</span></div></div></div></a></div></div><div style="flex-grow: 1;"></div><div class="style_buttons__16HgT"><div class="style_button-wrapper__2pKZZ"><button class="market-btn style_button__2DGnQ style_pill__I9z83 style_vertical__Ju5Fx" title="+6.5"><span class="style_label__2rlQp">+6.5</span><span class="style_price__3LrWW">1.970</span></button></div><div class="style_button-wrapper__2pKZZ"><button cla

In [21]:
print(bookies_info)

{}


In [22]:
def get_player_mean_per(last_games):
    per_values = []
    for index, game in last_games.iterrows():
        per_values.append((game['fgm'] * 85.910 + game['stl'] * 53.897 + game['fg3m'] * 51.757 + game['ftm'] * 46.845 + game['blk'] * 39.190 + game['oreb'] * 39.190 + game['ast'] * 34.677 + game['dreb'] * 14.707
                          - game['pf'] * 17.174 - (game['fta'] - game['ftm']) * 20.091 - (game['fga'] - game['fgm'])* 39.190 - game['tov'] * 53.897 ) * (1 / game['minutes']))
    if len(per_values) > 0:
        return mean(per_values)
    return 0

def get_team_per(lineup, team):
     # Getting players PER
        per_values = []
        for player in lineup:
            player = player.replace("'", '')
            try:
                player_object = season_games_plyrs.loc[(season_games_plyrs['team_id'] == team['id']) & ((season_games_plyrs['player_name'].str.contains(player)) | (season_games_plyrs['player_name'] == player) | 
                                                            (season_games_plyrs['player_name'].str.startswith(player[0]) & season_games_plyrs['player_name'].str.endswith(player.split(' ')[1])))].iloc[-1]
                last_ten_games = season_games_plyrs.loc[season_games_plyrs['player_id'] == player_object['player_id']].iloc[-10:]
                per_values.append(get_player_mean_per(last_ten_games))
            except Exception as e:
                print('Error when trying to get the games for {} of the {}: {}'.format(player, team['nickname'], e))
                continue

        if len(per_values) > 0:
            per = mean(per_values)
        else:
            per = 0
        
        return per

In [23]:
def get_games_lineups():
    url = "https://www.rotowire.com/basketball/nba-lineups.php"
    result = requests.get(url)
    doc = soup(result.text, "html.parser")
    games = doc.find_all("div", {"class": "lineup__box"})
    
    per_info = dict()
    
    for g in games:
        teams = [x.text for x in g.find_all("div", {"class": "lineup__abbr"})]
        if len(teams) == 0:
            continue
            
        away_abbr = teams[0]
        home_abbr = teams[1]
            
        home = teams_df.loc[teams_df['abbreviation'] == home_abbr].iloc[0]
        away = teams_df.loc[teams_df['abbreviation'] == away_abbr].iloc[0]

        away_lineup = g.find("ul", {"class": "lineup__list is-visit"})
        home_lineup = g.find("ul", {"class": "lineup__list is-home"})

        away_lineup = [x.text.split('\n')[-2] for x in away_lineup.find_all("li", {"class": "lineup__player"})][:5]
        home_lineup = [x.text.split('\n')[-2] for x in home_lineup.find_all("li", {"class": "lineup__player"})][:5]
        
        home_per = get_team_per(home_lineup, home)
        away_per = get_team_per(away_lineup, away)
        
        per_info[f"{home['id']}x{away['id']}"] = dict()
        per_info[f"{home['id']}x{away['id']}"][home['id']] = home_per
        per_info[f"{home['id']}x{away['id']}"][away['id']] = away_per
        
    return per_info

In [24]:
per_info = get_games_lineups()

Error when trying to get the games for Lamar Stevens of the Cavaliers: division by zero
Error when trying to get the games for P. Connaughton of the Bucks: single positional indexer is out-of-bounds


In [25]:
def get_team_made_conceded_pct(games, line):
    made = np.array(games['team_pts'])
    conceded = np.array(games['opp_pts'])
    totals = made + conceded
    greater = (totals > line).sum()
    pct = greater*100/len(totals)
    return pct

In [26]:
def get_team_elo(team):
    team = team.lower().replace(" ", "")
    if team == '76ers':
        team = 'sixers'
    contents = urllib.request.urlopen("https://projects.fivethirtyeight.com/complete-history-of-the-nba/data/{}.json".format(team)).read()
    contents = json.loads(contents)
    return contents['value'][-1]['y']

In [27]:
def get_team_previous_games(game_date, team, opp_id, line, scenario):  
    team_id = team['id']
    
    if scenario == 'H':
        per_info_key = f"{team_id}x{opp_id}"
    else:
        per_info_key = f"{opp_id}x{team_id}"
    
    response = hf.get_team_previous_games(season_games, team_id, game_date, season_split)
    if not response: return None
    
    home_previous_games, away_previous_games, previous_games, previous_season_games, home_previous_season_games, away_previous_season_games = response
    
    if len(previous_season_games.index) < min_games:
        return None
    
    last_n_games = previous_season_games.iloc[-n_last_games:,:]
    
    # Totals info
    totals_overall_pct = get_team_made_conceded_pct(previous_season_games.iloc[-totals_n_last_games:,:], line)
    if scenario == 'H':
        totals_ha_pct = get_team_made_conceded_pct(home_previous_season_games.iloc[-totals_n_last_games:,:], line)
    else:
        totals_ha_pct = get_team_made_conceded_pct(away_previous_season_games.iloc[-totals_n_last_games:,:], line)
    
    # Get last game ELO
    last_game_date = str(previous_season_games.iloc[-1,:]['date'])
    elo = get_team_elo(team['nickname'])
    
    # Last n games pct
    pct_last_n_games = hf.get_wl_pct(last_n_games)[0]
    
    # Getting Previous A x B Matchups
    last_matchups = previous_games[previous_games['opp_id'] == opp_id].iloc[-n_last_games:,:]
    
    # Getting player information
    team_per = per_info[per_info_key][team_id]
    
    # Season Win Percentage
    season_pct = hf.get_wl_pct(previous_season_games)[0]
    
    # Last n/2 games pct and Season H/A Win Percentage
    if scenario == 'H':
        ha_pct_last_n_games = hf.get_wl_pct(home_previous_season_games.iloc[-n_last_specific_games:,:])[0]
        ha_pct = hf.get_wl_pct(home_previous_season_games)[0]
    else:
        ha_pct_last_n_games = hf.get_wl_pct(away_previous_season_games.iloc[-n_last_specific_games:,:])[0]
        ha_pct = hf.get_wl_pct(away_previous_season_games)[0]
    
    # Matchup Win Percentage
    matchup_pct = hf.get_wl_pct(last_matchups)[0]
    
    # Calculating Current Streak
    streak = hf.current_streak(previous_season_games)
    
    stats_team = hf.get_team_stats (last_n_games, season_pct, team_per, elo, matchup_pct, ha_pct, streak, pct_last_n_games, ha_pct_last_n_games, totals_overall_pct, totals_ha_pct)
    
    return stats_team

In [28]:
for fixture in fixtures:
    game_date = np.datetime64(dt.datetime.utcfromtimestamp(fixture['timestamp']).replace(tzinfo=from_zone).astimezone(to_zone).strftime('%Y-%m-%d %H:%M:%S'))
    home_name, away_name = fixture['teams']['home']['name'], fixture['teams']['away']['name']
    
    home = teams_df.loc[teams_df['full_name'] == home_name].iloc[0]
    away = teams_df.loc[teams_df['full_name'] == away_name].iloc[0]
    
    print('-'*10)
    
    try:
        bookie_info = bookies_info[f"{home['id']}x{away['id']}"]

        line = bookie_info['totals_line']
        over_odds = bookie_info['totals_over_odds']
        under_odds = bookie_info['totals_under_odds']
        home_odds = bookie_info['home_ml_odds']
        away_odds = bookie_info['away_ml_odds']
        spread_line = bookie_info['spread_line']
    except:
        print(f"Game {away_name} @ {home_name}")
        line = float(input("Type the totals line: "))
        over_odds = under_odds = 1.9
        home_odds = float(input("Type the home odds: "))
        away_odds = float(input("Type the away_odds: "))
        spread_line = float(input("Type the spread line: "))
#         bookie_info = input('Type the line, over_odds, under_odds, home_odds and the away odds in the format "X;X;X;X;X". If you want to continue, type "-1": ')
#         if bookie_info == '-1': continue
#         line, over_odds, under_odds, home_odds, away_odds = map(float, bookie_info.split(';'))
        
    stats_a = get_team_previous_games(game_date, home, away['id'], line, 'H')
    
    if not stats_a: continue

    home_overall_pct, home_ha_pct = stats_a[-2], stats_a[-1]

    stats_b = get_team_previous_games(game_date, away, home['id'], line, 'A')
    if not stats_b: continue

    away_overall_pct, away_ha_pct = stats_b[-2], stats_b[-1]
    
    game_stats = stats_a[:-2] + stats_b[:-2]
    
    home_pred_pts, away_pred_pts = model.predict([game_stats])[0]
    pred_pts_sum = home_pred_pts + away_pred_pts
    
    print("{} had their totals above {} in {:.2f}% of the last {} games".format(home_name, line, home_overall_pct, n_last_games))
    print("At home, {} had their totals above {} in {:.2f}% of the last {} games".format(home_name, line, home_ha_pct, n_last_games))
    
    print("\n{} had their totals above {} in {:.2f}% of the last {} games".format(away_name, line, away_overall_pct, n_last_games))
    print("On the road, {} had their totals above {} in {:.2f}% of the last {} games".format(away_name, line, away_ha_pct, n_last_games))
    
    pct_list = [home_overall_pct, home_ha_pct, away_overall_pct, away_ha_pct]
    values_greater = sum(x > totals_max_threshold for x in pct_list)
    values_lower = sum(x < totals_min_threshold for x in pct_list)
    over = values_greater > 3 and pred_pts_sum > line
    under = values_lower > 3 and pred_pts_sum < line
    print(colored('TOTALS BET:', 'blue'))
    if (over or under) and abs(line - pred_pts_sum) > 2:
        print(colored(f"GOOD BET: {'OVER' if over else 'UNDER'} @ {over_odds if over else under_odds}", 'green'))
    else:
        print(colored(f"BAD BET: {'OVER' if over else 'UNDER'} @ {over_odds if over else under_odds}", 'red'))
        
    if home_pred_pts > away_pred_pts:
        prediction = home_name
        selected_odds = home_odds
    else:
        prediction = away_name
        selected_odds = away_odds
        
    underdog = home_name if home_odds > away_odds else away_name
        
    print(colored('\nMONEYLINE/SPREAD BET:', 'blue'))
    if abs(home_pred_pts - away_pred_pts) < 3 and spread_line > 9:
        print(colored(f"GOOD BET: {underdog} +{spread_line}", 'green'))
    elif abs(home_pred_pts - away_pred_pts) < 3 or selected_odds > min_threshold or selected_odds < max_threshold:
        print(colored(f"BAD BET: {prediction} @ {selected_odds}", 'red'))
    else:
        print(colored(f"GOOD BET: {prediction} @ {selected_odds}", 'green'))

----------
Game Atlanta Hawks @ Cleveland Cavaliers
Type the totals line: 224.5
Type the home odds: 1.76
Type the away_odds: 2.18
Type the spread line: 2.5
Cleveland Cavaliers had their totals above 224.5 in 46.67% of the last 10 games
At home, Cleveland Cavaliers had their totals above 224.5 in 57.14% of the last 10 games

Atlanta Hawks had their totals above 224.5 in 73.33% of the last 10 games
On the road, Atlanta Hawks had their totals above 224.5 in 85.71% of the last 10 games
TOTALS BET:
BAD BET: UNDER @ 1.9

MONEYLINE/SPREAD BET:
BAD BET: Cleveland Cavaliers @ 1.76
----------
Game Orlando Magic @ Indiana Pacers
Type the totals line: 226.5
Type the home odds: 1.39
Type the away_odds: 3.21
Type the spread line: 6.5
Indiana Pacers had their totals above 226.5 in 60.00% of the last 10 games
At home, Indiana Pacers had their totals above 226.5 in 62.50% of the last 10 games

Orlando Magic had their totals above 226.5 in 40.00% of the last 10 games
On the road, Orlando Magic had their

KeyboardInterrupt: Interrupted by user